In [5]:
# # https://deeplearningcourses.com/c/deep-learning-advanced-nlp
# get the data at: http://www.manythings.org/anki/
from __future__ import print_function, division
from builtins import range, input
# Note: you may need to update your version of future
# sudo pip install -U future

import os, sys
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

import keras.backend as K
if len(K.tensorflow_backend._get_available_gpus()) > 0:
  from keras.layers import CuDNNLSTM as LSTM
  from keras.layers import CuDNNGRU as GRU

In [6]:
# some config
BATCH_SIZE = 64  # Batch size for training.
EPOCHS = 100  # Number of epochs to train for.
LATENT_DIM = 256  # Latent dimensionality of the encoding space.
#NUM_SAMPLES = 10000  # Number of samples to train on.
MAX_SEQUENCE_LENGTH = 100
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 50

In [7]:
# Where we will store the data
input_texts = [] # sentence in original language
target_texts = [] # sentence in target language
target_texts_inputs = [] # sentence in target language offset by 1

In [8]:
for line in open('D:/notebooks/udemy/deeplearning-advancedNLP/machine_learning_examples/large_files/hin.txt',encoding='utf-8'):
    if '\t' not in line:
         continue

    #split up the input and translation
    input_text, translation = line.rstrip().split('\t')
    '''print(input_text," ",translation)
    print()'''
    target_text = translation + ' <eos>'
    target_text_input = '<sos> ' + translation
    input_texts.append(input_text)
    target_texts.append(target_text)
    target_texts_inputs.append(target_text_input)
print("num samples:", len(input_texts))

num samples: 2863


In [9]:
input_texts[:10]

['Wow!',
 'Help!',
 'Jump.',
 'Jump.',
 'Jump.',
 'Hello!',
 'Hello!',
 'Cheers!',
 'Cheers!',
 'Got it?']

In [10]:
target_texts[:10]

['वाह! <eos>',
 'बचाओ! <eos>',
 'उछलो. <eos>',
 'कूदो. <eos>',
 'छलांग. <eos>',
 'नमस्ते। <eos>',
 'नमस्कार। <eos>',
 'वाह-वाह! <eos>',
 'चियर्स! <eos>',
 'समझे कि नहीं? <eos>']

In [11]:
target_texts_inputs[:10]

['<sos> वाह!',
 '<sos> बचाओ!',
 '<sos> उछलो.',
 '<sos> कूदो.',
 '<sos> छलांग.',
 '<sos> नमस्ते।',
 '<sos> नमस्कार।',
 '<sos> वाह-वाह!',
 '<sos> चियर्स!',
 '<sos> समझे कि नहीं?']

In [12]:
#tokenize the inputs
tokenizer_inputs=Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer_inputs.fit_on_texts(input_texts)
input_sequences=tokenizer_inputs.texts_to_sequences(input_texts)
max_len_input=max(len(doc) for doc in input_sequences)
word2idx_inputs=tokenizer_inputs.word_index
print(len(word2idx_inputs))

2402


In [13]:
#tokenize the outputs
tokenizer_outputs=Tokenizer(num_words=MAX_NUM_WORDS,filters='')
tokenizer_outputs.fit_on_texts(target_texts+target_texts_inputs)
target_sequences=tokenizer_outputs.texts_to_sequences(target_texts)
target_sequences_inputs=tokenizer_outputs.texts_to_sequences(target_texts_inputs)
word2idx_outputs=tokenizer_outputs.word_index
max_len_output=max(len(doc) for doc in target_sequences)
num_words_output=len(word2idx_outputs)+1

In [14]:
num_words_output

3156

In [15]:
word2idx_outputs

{'<eos>': 1,
 '<sos>': 2,
 'है।': 3,
 'में': 4,
 'नहीं': 5,
 'मैं': 6,
 'वह': 7,
 'से': 8,
 'के': 9,
 'मुझे': 10,
 'को': 11,
 'हैं।': 12,
 'की': 13,
 'बहुत': 14,
 'का': 15,
 'हूँ।': 16,
 'पर': 17,
 'उसने': 18,
 'तुम': 19,
 'हो': 20,
 'था।': 21,
 'एक': 22,
 'यह': 23,
 'क्या?': 24,
 'क्या': 25,
 'कर': 26,
 'लिए': 27,
 'है': 28,
 'मेरे': 29,
 'है?': 30,
 'कि': 31,
 'उसे': 32,
 'इस': 33,
 'और': 34,
 'मैंने': 35,
 'भी': 36,
 'मेरी': 37,
 'अपने': 38,
 'रहा': 39,
 'ने': 40,
 'पास': 41,
 'तो': 42,
 'करने': 43,
 'ही': 44,
 'अपनी': 45,
 'कल': 46,
 'हम': 47,
 'काम': 48,
 'गया।': 49,
 'उसके': 50,
 'उसकी': 51,
 'कोई': 52,
 'तुम्हें': 53,
 'तुम्हारे': 54,
 'उस': 55,
 'मेरा': 56,
 'घर': 57,
 'करना': 58,
 'सकते': 59,
 'साथ': 60,
 'थी।': 61,
 'समय': 62,
 'बात': 63,
 'गया': 64,
 'पता': 65,
 'रही': 66,
 'हो?': 67,
 'बजे': 68,
 'आप': 69,
 'आज': 70,
 'उसको': 71,
 'कुछ': 72,
 'यहाँ': 73,
 'तक': 74,
 'तुम्हे': 75,
 'चाहिए।': 76,
 'हैं': 77,
 'लगता': 78,
 'किताब': 79,
 'मुझसे': 80,
 'अच्छा': 81,
 'अपना': 82,


In [16]:
#pad the sequences
encoder_inputs=pad_sequences(input_sequences,maxlen=max_len_input,padding='post')
print(encoder_inputs.shape)
print(encoder_inputs[0])
decoder_inputs=pad_sequences(target_sequences_inputs,maxlen=max_len_output,padding='post')
print(decoder_inputs.shape)
print(decoder_inputs[0])
decoder_target=pad_sequences(target_sequences,maxlen=max_len_output,padding='post')
print(decoder_target.shape)
print(decoder_target[0])

(2863, 22)
[1279    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
(2863, 26)
[   2 1500    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
(2863, 26)
[1500    1    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


In [17]:
word2vec={}
with open("D:/notebooks/glove.6B.50d.txt",encoding='utf-8') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vec=np.asarray(values[1:],dtype='float32')
        word2vec[word]=vec

In [18]:
len(word2vec)

400000

In [19]:
# prepare embedding matrix
print('Filling pre-trained embeddings...')
num_words = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word2idx_inputs.items():
    if i < MAX_NUM_WORDS:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
         # words not found in embedding index will be all zeros.
         embedding_matrix[i] = embedding_vector

Filling pre-trained embeddings...


In [20]:
#creating embedding layer
embedding_layer=Embedding(num_words,EMBEDDING_DIM,weights=[embedding_matrix],input_length=max_len_input,trainable=False)

In [21]:
# create targets, since we cannot use sparse
# categorical cross entropy when we have sequences
decoder_targets_one_hot = np.zeros((len(input_texts),max_len_output,num_words_output),dtype='float32')
# assign the values
for i, d in enumerate(decoder_target):
    for t, word in enumerate(d):
        decoder_targets_one_hot[i, t, word] = 1

In [22]:
#build the model
encoder_inputs_placeholder=Input((max_len_input,))
x=embedding_layer(encoder_inputs_placeholder)
encoder=LSTM(LATENT_DIM,return_sequences=False,return_state=True)
encoder_outputs,h,c=encoder(x)

In [23]:
encoder_states=[h,c]
decoder_inputs_placeholder=Input((max_len_output,))
decoder_embeddings=Embedding(num_words_output,EMBEDDING_DIM,trainable=True)
decoder_inputs_x=decoder_embeddings(decoder_inputs_placeholder)
decoder=LSTM(LATENT_DIM,return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder(decoder_inputs_x,initial_state=encoder_states)

In [24]:
decoder_dense=Dense(num_words_output,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

In [26]:
model=Model([encoder_inputs_placeholder,decoder_inputs_placeholder],decoder_outputs)

In [27]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [28]:
model.load_weights("s2s.h5")

In [ ]:
r = model.fit([encoder_inputs, decoder_inputs], decoder_targets_one_hot,
  batch_size=BATCH_SIZE,
  epochs=EPOCHS,
  validation_split=0.2,
)

Train on 11452 samples, validate on 2863 samples
Epoch 1/100
11452/11452 [==============================] - 121s 11ms/step - loss: 1.9595 - acc: 0.7165 - val_loss: 1.6164 - val_acc: 0.7464
Epoch 2/100
11452/11452 [==============================] - 123s 11ms/step - loss: 1.5370 - acc: 0.7549 - val_loss: 1.4238 - val_acc: 0.7634
Epoch 3/100
11452/11452 [==============================] - 122s 11ms/step - loss: 1.3594 - acc: 0.7684 - val_loss: 1.2622 - val_acc: 0.7748
Epoch 4/100
11200/11452 [============================>.] - ETA: 2s - loss: 1.2294 - acc: 0.7779

In [29]:
encoder_model = Model(encoder_inputs_placeholder, encoder_states)

decoder_state_input_h = Input(shape=(LATENT_DIM,))
decoder_state_input_c = Input(shape=(LATENT_DIM,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [30]:
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embeddings(decoder_inputs_single)
decoder_outputs, h, c = decoder(
  decoder_inputs_single_x,
  initial_state=decoder_states_inputs
)

In [31]:
decoder_states = [h, c]
# decoder_states = [h] # gru
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
  [decoder_inputs_single] + decoder_states_inputs, 
  [decoder_outputs] + decoder_states
)

In [57]:
print(decoder_outputs.shape)
print(h.shape)
print(c.shape)

(?, 1, 3156)
(?, 256)
(?, 256)


In [32]:
idx2word_trans = {v:k for k, v in word2idx_outputs.items()}
idx2word_eng = {v:k for k, v in word2idx_inputs.items()}

In [34]:
def decode_sequence(input_seq):
  # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

  # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))

  # Populate the first character of target sequence with the start character.
  # NOTE: tokenizer lower-cases all words
    target_seq[0, 0] = word2idx_outputs['<sos>']

  # if we get this we break
    eos = word2idx_outputs['<eos>']

  # Create the translation
    output_sentence = []
    for _ in range(max_len_output):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
    # output_tokens, h = decoder_model.predict(
    #     [target_seq] + states_value
    # ) # gru

    # Get next word
        idx = np.argmax(output_tokens[0, 0, :])

    # End sentence of EOS
        if eos == idx:
              break

        word = ''
        if idx > 0:
            word = idx2word_trans[idx]
            output_sentence.append(word)

    # Update the decoder input
    # which is just the word just generated
        target_seq[0, 0] = idx

    # Update states
        states_value = [h, c]
    # states_value = [h] # gru

    return output_sentence



while True:
  # Do some test translations
    i = np.random.choice(len(input_texts))
    input_seq = encoder_inputs[i:i+1]
    translation = decode_sequence(input_seq)
    print('-')
    print('Input:', input_texts[i])
    print('Translation:', ' '.join(translation))
    ans = input("Continue? [Y/n]")
    if ans and ans.lower().startswith('n'):
        break

-
Input: Are you tired?
Translation: थक गए हो क्या?
Continue? [Y/n]Y
-
Input: I know what that is.
Translation: मुझे पता है कि वह चीज़ क्या है।
Continue? [Y/n]n


In [35]:
from nltk.translate.bleu_score import sentence_bleu
ypred=[]
bleu_score=0.0
for i in range(len(input_texts)):
    input_seq=encoder_inputs[i:i+1]
    translation=decode_sequence(input_seq)
    ypred.append(translation)

In [36]:
for i in range(len(input_texts)):
    yact=target_texts[i].split(" ")[:-1]
    bleu_score+=sentence_bleu([yact],ypred[i])
print(bleu_score/len(input_texts))

C:\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


0.6870082578051827


In [62]:
# Save model
model.save('s2s.h5')

C:\Anaconda3\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_4:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [51]:
#build the model
encoder2=GRU(LATENT_DIM,return_sequences=False,return_state=True)
encoder2_outputs,h2=encoder2(x)
encoder2_states=[h2]
decoder2=GRU(LATENT_DIM,return_sequences=True,return_state=True)
decoder2_outputs,_=decoder2(decoder_inputs_x,initial_state=encoder2_states)

In [52]:
decoder2_outputs=decoder_dense(decoder2_outputs)
model2=Model([encoder_inputs_placeholder,decoder_inputs_placeholder],decoder2_outputs)
model2.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [53]:
r = model2.fit([encoder_inputs, decoder_inputs], decoder_targets_one_hot,
  batch_size=BATCH_SIZE,
  epochs=20,
  validation_split=0.2,
)

Train on 2290 samples, validate on 573 samples
Epoch 1/20
2290/2290 [==============================] - 23s 10ms/step - loss: 1.8512 - acc: 0.7488 - val_loss: 2.5928 - val_acc: 0.6292
Epoch 2/20
2290/2290 [==============================] - 25s 11ms/step - loss: 1.3056 - acc: 0.7886 - val_loss: 2.5808 - val_acc: 0.6350
Epoch 3/20
2290/2290 [==============================] - 25s 11ms/step - loss: 1.1529 - acc: 0.8003 - val_loss: 2.5777 - val_acc: 0.6459
Epoch 4/20
2290/2290 [==============================] - 24s 11ms/step - loss: 1.0054 - acc: 0.8129 - val_loss: 2.6090 - val_acc: 0.6533
Epoch 5/20
2290/2290 [==============================] - 24s 10ms/step - loss: 0.8867 - acc: 0.8243 - val_loss: 2.7029 - val_acc: 0.6543
Epoch 6/20
2290/2290 [==============================] - 24s 10ms/step - loss: 0.7902 - acc: 0.8348 - val_loss: 2.7494 - val_acc: 0.6560
Epoch 7/20
2290/2290 [==============================] - 24s 11ms/step - loss: 0.7055 - acc: 0.8478 - val_loss: 2.7819 - val_acc: 0.6574
E

In [74]:
encoder2_model = Model(encoder_inputs_placeholder, encoder2_states)
decoder2_states_inputs = [decoder_state_input_h]
decoder2_outputs2, h3= decoder2(decoder_inputs_single_x,initial_state=decoder2_states_inputs)
decoder2_states = [h3]
# decoder_states = [h] # gru
decoder2_outputs2 = decoder_dense(decoder2_outputs2)
decoder2_model = Model([decoder_inputs_single] + decoder2_states_inputs,[decoder2_outputs2] + decoder2_states)

In [80]:
def decode_sequence2(input_seq):
  # Encode the input as state vectors.
    states_value = encoder2_model.predict(input_seq)
    print(states_value.shape)

  # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))

  # Populate the first character of target sequence with the start character.
  # NOTE: tokenizer lower-cases all words
    target_seq[0, 0] = word2idx_outputs['<sos>']

  # if we get this we break
    eos = word2idx_outputs['<eos>']

  # Create the translation
    output_sentence = []
    for _ in range(max_len_output):
        output_tokens, h= decoder2_model.predict([target_seq] + [states_value])
    # output_tokens, h = decoder_model.predict(
    #     [target_seq] + states_value
    # ) # gru

    # Get next word
        idx = np.argmax(output_tokens[0, 0, :])

    # End sentence of EOS
        if eos == idx:
              break

        word = ''
        if idx > 0:
            word = idx2word_trans[idx]
            output_sentence.append(word)

    # Update the decoder input
    # which is just the word just generated
        target_seq[0, 0] = idx

    # Update states
        states_value = h
    # states_value = [h] # gru

    return output_sentence



while True:
  # Do some test translations
    i = np.random.choice(len(input_texts))
    input_seq = encoder_inputs[i:i+1]
    translation = decode_sequence2(input_seq)
    print('-')
    print('Input:', input_texts[i])
    print('Translation:', ' '.join(translation))
    ans = input("Continue? [Y/n]")
    if ans and ans.lower().startswith('n'):
        break

(1, 256)
-
Input: India is the seventh largest country in the world.
Translation: फ़्रान्स जापान में से एक नदी को भूत है।
Continue? [Y/n]Y
(1, 256)
-
Input: I don't want to buy this kind of sofa.
Translation: मुझे इस तरह का सोफ़ा नहीं खरीदना है।
Continue? [Y/n]Y
(1, 256)
-
Input: There is a limit to how much one can tolerate.
Translation: यह समस्या को सुलझाना बहुत कठिन है।
Continue? [Y/n]Y
(1, 256)
-
Input: When he is in trouble, he always turns to his sister for help.
Translation: वह मेरे बजाय वहां गया।
Continue? [Y/n]Y
(1, 256)
-
Input: She is two years younger than me.
Translation: वह मेरे लिए एक छोटी है।
Continue? [Y/n]Y
(1, 256)
-
Input: I don't like to drink coffee.
Translation: मुझे अभी खाना खाने के लिए पसंद है।
Continue? [Y/n]Y
(1, 256)
-
Input: Show me another example.
Translation: मेरे लिए एक बीटल्स का जवाब है।
Continue? [Y/n]Y
(1, 256)
-
Input: Only a few people listened to him.
Translation: वह आदमी दिनों से टॉम ने वह लोगों को भी नहीं।
Continue? [Y/n]n


In [81]:
from nltk.translate.bleu_score import sentence_bleu
ypred=[]
bleu_score=0.0
for i in range(len(input_texts)):
    input_seq=encoder_inputs[i:i+1]
    translation=decode_sequence2(input_seq)
    ypred.append(translation)

(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(

(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(

(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(

(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)
(1, 256)


In [82]:
for i in range(len(input_texts)):
    yact=target_texts[i].split(" ")[:-1]
    bleu_score+=sentence_bleu([yact],ypred[i])
print(bleu_score/len(input_texts))

C:\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Anaconda3\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


0.2714373702704523
